In [1]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
monkey.patch_all(thread=False)
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = Trueee
server.serve_forever()
#http://127.0.0.2:5000/

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['pymongo.ssl_support (C:\\ProgramData\\Anaconda3\\lib\\site-packages\\pymongo\\ssl_support.py)', 'urllib3.util (C:\\ProgramData\\Anaconda3\\lib\\site-packages\\urllib3\\util\\__init__.py)', 'urllib3.contrib.pyopenssl (C:\\ProgramData\\Anaconda3\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)']. 
  """Entry point for launching an IPython kernel.


{'title': 'admin|api keys', 'params': {'alphavantage': 'AWT27M4Y1HHNIGXL', 'ff': 'rr'}}


127.0.0.1 - - [2020-03-03 10:49:35] "GET /admin/api_keys HTTP/1.1" 200 6482 0.061431
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.001998
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/css/template.css HTTP/1.1" 200 16644 0.001996
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/css/mycss.css HTTP/1.1" 200 2076 0.002032
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88469 0.001952
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 200 21585 0.001995
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60337 0.001994
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/js/table_search_filter.js HTTP/1.1" 200 931 0.000998
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/js/template.js HTTP/1.1" 200 1194 0.000997
127.0.0.1 - - [2020-03-03 10:49:35] "GET /static/vendor/fontawesome-free/webfont

{'title': 'admin|api keys', 'params': {'alphavantage': 'AWT27M4Y1HHNIGXL', 'ff': ''}}
{'title': 'admin|api keys', 'params': {'alphavantage': 'AWT27M4Y1HHNIGXL', 'ff': ''}}


127.0.0.1 - - [2020-03-03 10:49:44] "GET /admin/api_keys HTTP/1.1" 200 6480 0.018370
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.002994
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/css/template.css HTTP/1.1" 200 16644 0.001993
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/css/mycss.css HTTP/1.1" 200 2076 0.000997
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88469 0.000997
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 200 21585 0.001994
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60337 0.003000
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/js/table_search_filter.js HTTP/1.1" 200 931 0.000997
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/js/template.js HTTP/1.1" 200 1194 0.002037
127.0.0.1 - - [2020-03-03 10:49:44] "GET /static/vendor/fontawesome-free/webfont

In [ ]:
import mygit as mg
from pathlib import Path
import os
folder=os.path.abspath('{}/..'.format(os.getcwd()))
#folder=Path(folder).parent
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
#gt.pyignore()
#gt.pull()
gt.push_commit(git_url,commit_msg='update')

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\src")

In [ ]:
from commons import convert_file_size,init_db_manager
from jobs import *
#a=job_sectors(init_db_manager())

In [ ]:
a=job_sl_bonds(init_db_manager())

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

In [ ]:
from external_sources import mosex
import pandas as pd
a=mosex()